In [13]:
# Import basic libraries
from palmerpenguins import load_penguins
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")
import time
from sklearn.preprocessing import LabelEncoder
import os
os.chdir("/Users/hoangthuyduongvu/Desktop/FuzzSIM")


In [14]:
# Import personalized libraries
from fuzz.src.capacity import *
from fuzz.choquet.choquet import *
from fuzz.src.norm import *
from fuzz.src.knn import KNNFuzz
from fuzz.src.sim import S1, S2, S3
from fuzz.optim import *
from fuzz.utils import *
from fuzz.eval import leave_one_out
from fuzz.dataloader import *
from fuzz.choquet.d_choquet import *

# BASIC

In [32]:
def monotonic_check_unit(X: Capacity, Y: Capacity) -> bool: 
    # Check if set X is in set Y
    x_keys = X.X
    y_keys = Y.X

    if set(x_keys).issubset(set(y_keys)):
        # print(f"Set {x_keys} is a subset of set {y_keys}.")
        # Check if the values of X are less than or equal to the values of Y
        if len(x_keys) == len(y_keys):
            return True 
        elif len(x_keys) < len(y_keys):
            if X.mu <= Y.mu:
                return True
    elif set(y_keys).issubset(set(x_keys)):
        # print(f"Set {y_keys} is a subset of set {x_keys}.")
        # Check if the values of Y are less than or equal to the values of X
        if len(x_keys) == len(y_keys):
            return True 
        elif len(x_keys) > len(y_keys):
            if Y.mu <= X.mu:
                return True
    return False

In [35]:
print(monotonic_check_unit(
    X = Capacity(X=[1,2,3], mu=0.4),
    Y = Capacity(X=[2,3], mu=0.1)
))
print(monotonic_check_unit(
    X = Capacity(X=[1,2,3], mu=0.04),
    Y = Capacity(X=[2,3], mu=0.1)
))
print(monotonic_check_unit(
    X = Capacity(X=[1,3], mu=0.04),
    Y = Capacity(X=[2,3], mu=0.1)
))

True
False
False
